In [19]:
#상품정보 크롤링
def get_items():
    driver.find_element(By.XPATH, f'//*[@id="Contents"]/ul[{i}]/li[{j}]/div/div/a').click()
    time.sleep(1) 
    brand = driver.find_element(By.CLASS_NAME, "prd_brand").text
    name = driver.find_element(By.CLASS_NAME, "prd_name").text
    price = driver.find_element(By.CLASS_NAME, "price-1").text
    sale_price = driver.find_element(By.CLASS_NAME, "price-2").text
    picture = driver.find_element(By.XPATH, '//*[@id="mainImg"]').get_attribute('src')
    url = driver.find_element(By.XPATH, '/html/head/meta[8]').get_attribute('content')

    driver.find_element(By.CSS_SELECTOR, "#buyInfo > a").click()
    time.sleep(1)
    volume = driver.find_element(By.XPATH,'//*[@id="artcInfo"]/dl[2]/dd').text
    skin_type = driver.find_element(By.XPATH,'//*[@id="artcInfo"]/dl[3]/dd').text
    ingredient = driver.find_element(By.XPATH,'//*[@id="artcInfo"]/dl[8]/dd').text

    print(brand)
    print(name)
    print(price) 
    print(sale_price)
    print(picture)
    print(url)
    print(volume)
    print(skin_type)
    print(ingredient)
    
    
#리뷰 크롤링
def get_review():
    driver.find_element(By.CSS_SELECTOR, "#reviewInfo > a").click() #리뷰버튼 클릭
    time.sleep(2) 
    driver.find_element(By.XPATH, '//*[@id="gdasSort"]/li[3]/a').click() #최신순 클릭
    time.sleep(1) 
    driver.find_element(By.CSS_SELECTOR, "#searchType_3").click() #체험단 체크 해제 
    time.sleep(1)
    
    review_count = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a/span').text 
    review_count = int(review_count.replace('(', '').replace(')', '').replace(',', ''))

    prod_class = driver.find_element(By.CLASS_NAME, 'prd_option_box box_select filter')
    
    all_reviews = []
    #리뷰가 100이 안넘는다면 
    if (review_count < 100):
        p = int(str(review_count)[0]) #십의자리 ex)리뷰수가 68개면 6만가져옴 >> 마지막페이지 알기위해 
        o = int(str(review_count)[1]) # 8 
        k = 1
        while k <= p:
            reviews = [] #수정
            for r in range(1, 11):
                #마지막 페이지일때 
                if k == p:
                    for r in range(1, o+1):
                        try:
                            review = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{r}]/div[2]/div[3]').text
                            reviews.append([review])
                        except:
                            break
                    all_reviews.extend(reviews)
                    print(all_reviews)
                    Lotion_dataset.loc[len(Lotion_dataset)] = [brand, name,price,sale_price,picture,url,volume,skin_type,ingredient, all_reviews]    
                else:
                    try:
                        review = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{r}]/div[2]/div[3]').text
                        reviews.append([review])
                    except:
                        break
                all_reviews.extend(reviews)
                
            #다음 페이지 버튼 누르기
            if element_exists(prod_class) == True:
                button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{k}]').click()   
                k += 1
                time.sleep(2)
            else:
                button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{k}]').click()
                k += 1
                time.sleep(2)
        Lotion_dataset.loc[len(Lotion_dataset)] = [brand, name,price,sale_price,picture,url,volume,skin_type,ingredient, all_reviews]
        print(all_reviews)
        
    # 리뷰가 100개 초과일 때    
    else:
        k = 1
        while k < 10: # k가 9까지일때가 10페이지임
            reviews = []
            for r in range(1, 11):
                try:
                    review = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{r}]/div[2]/div[3]').text
                    reviews.append([review])
                except:
                    break
            all_reviews.extend(reviews)
            
            if element_exists(prod_class) == True:
                button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{k}]').click()   
                k += 1
                time.sleep(2)
            else:
                button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{k}]').click()
                k += 1
                time.sleep(2)
        Lotion_dataset.loc[len(Lotion_dataset)] = [brand, name,price,sale_price,picture,url,volume,skin_type,ingredient, all_reviews]  
        print(all_reviews)

In [20]:
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import NoSuchElementException

import numpy as np 
import pandas as pd
import time
import random

from bs4 import BeautifulSoup


chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ['enable-logging'])
chrome_options.add_argument("--disable-blink-features-AutomationControlled")

driver = webdriver.Chrome(options=chrome_options)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
# 브라우저 창 크기 설정
driver.set_window_size(1920, 1080)

Lotion_dataset = pd.DataFrame(columns= ['brand', 'name','price','sale_price','picture','url','volume','skin_type','ingredient','review'])
Lotion_dataset

def element_exists(by, value):
    try:
        driver.find_element(by, value)
        return True
    except NoSuchElementException:
        return False
#상품페이지 
for page_num in range(1,6):
    olive_url = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010016&fltDispCatNo=&prdSort=01&pageIdx={page_num}&rowsPerPage=48&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=1&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat100000100010016_Small&amplitudePageGubun=&t_page=&t_click=&midCategory=%EB%A1%9C%EC%85%98&smallCategory=%EC%A0%84%EC%B2%B4&checkBrnds=&lastChkBrnd="
    driver.get(olive_url)
    time.sleep(1)
    
    #5페이지는 42개상품밖에 없음 
    if page_num == 5:
        for i in range(2, 13):
            #마지막 페이지의 마지막 줄 
            if i == 12: #//*[@id="Contents"]/ul[12]/li[2]/div/div/a
                for j in range(1,3):
                    
                    driver.find_element(By.XPATH, f'//*[@id="Contents"]/ul[{i}]/li[{j}]/div/div/a').click()
                    time.sleep(1) 
                    brand = driver.find_element(By.CLASS_NAME, "prd_brand").text
                    name = driver.find_element(By.CLASS_NAME, "prd_name").text
                    if (element_exists(By.CLASS_NAME, "price-1") == True):
                        price = driver.find_element(By.CLASS_NAME, "price-1").text
                        sale_price = driver.find_element(By.CLASS_NAME, "price-2").text
                    else: 
                        sale_price = driver.find_element(By.CLASS_NAME, "price-2").text
                    picture = driver.find_element(By.XPATH, '//*[@id="mainImg"]').get_attribute('src')
                    url = driver.find_element(By.XPATH, '/html/head/meta[8]').get_attribute('content')
                    
                    driver.find_element(By.CSS_SELECTOR, "#buyInfo > a").click()
                    time.sleep(1)
                    volume = driver.find_element(By.XPATH,'//*[@id="artcInfo"]/dl[2]/dd').text
                    skin_type = driver.find_element(By.XPATH,'//*[@id="artcInfo"]/dl[3]/dd').text
                    ingredient = driver.find_element(By.XPATH,'//*[@id="artcInfo"]/dl[8]/dd').text
                
                    print(brand)
                    print(name)
                    print(price) 
                    print(sale_price)
                    print(picture)
                    print(url)
                    print(volume)
                    print(skin_type)
                    print(ingredient)
                    
                    #리뷰 크롤링
                    driver.find_element(By.CSS_SELECTOR, "#reviewInfo > a").click() #리뷰버튼 클릭
                    time.sleep(2) 
                    driver.find_element(By.XPATH, '//*[@id="gdasSort"]/li[3]/a').click() #최신순 클릭
                    time.sleep(1) 
                    driver.find_element(By.CSS_SELECTOR, "#searchType_3").click() #체험단 체크 해제 
                    time.sleep(1)
                    
                    review_count = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a/span').text 
                    review_count = int(review_count.replace('(', '').replace(')', '').replace(',', ''))
            
                    all_reviews = []
                    #리뷰가 100이 안넘는다면 
                    if (review_count < 100):
                        p = int(str(review_count)[0]) #십의자리 ex)리뷰수가 68개면 6만가져옴 >> 마지막페이지 알기위해 
                        o = int(str(review_count)[1]) # 8 
                        k = 1
                        while k <= p:
                            reviews = [] #수정
                            for r in range(1, 11):
                                #마지막 페이지일때 
                                if k == p:
                                    for r in range(1, o+1):
                                        try:
                                            review = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{r}]/div[2]/div[3]').text
                                            reviews.append([review])
                                        except:
                                            break
                                    all_reviews.extend(reviews)
                                    print(all_reviews)
                                    Lotion_dataset.loc[len(Lotion_dataset)] = [brand, name,price,sale_price,picture,url,volume,skin_type,ingredient, all_reviews] 
                                    break  
                                
                            try:
                                review = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{r}]/div[2]/div[3]').text
                                reviews.append([review])
                            except:
                                break
                            all_reviews.extend(reviews)
                                
                            #다음 페이지 버튼 누르기
                            if (element_exists(By.XPATH, '//*[@id="sel_option_box"]') == True):
                                button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{k}]').click()   
                                k += 1
                                time.sleep(2)
                            else:
                                button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{k}]').click()
                                k += 1
                                time.sleep(2)
                        Lotion_dataset.loc[len(Lotion_dataset)] = [brand, name,price,sale_price,picture,url,volume,skin_type,ingredient, all_reviews]
                        print(all_reviews)
                        driver.back()
                    # 리뷰가 100개 초과일 때    
                    else:
                        k = 1
                        while k < 10: # k가 9까지일때가 10페이지임
                            reviews = []
                            for r in range(1, 11):
                                try:
                                    review = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{r}]/div[2]/div[3]').text
                                    reviews.append([review])
                                except:
                                    break
                            all_reviews.extend(reviews)
                            
                            if (element_exists(By.XPATH, '//*[@id="sel_option_box"]') == True):
                                button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{k}]').click()   
                                k += 1
                                time.sleep(2)
                            else:
                                button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{k}]').click()
                                k += 1
                                time.sleep(2)
                        Lotion_dataset.loc[len(Lotion_dataset)] = [brand, name,price,sale_price,picture,url,volume,skin_type,ingredient, all_reviews]  
                        print(all_reviews) 
                        break        
            #1-11번째 줄         
            for j in range(1, 4):  
                driver.find_element(By.XPATH, f'//*[@id="Contents"]/ul[{i}]/li[{j}]/div/div/a').click()
                time.sleep(1) 
                brand = driver.find_element(By.CLASS_NAME, "prd_brand").text
                name = driver.find_element(By.CLASS_NAME, "prd_name").text
                if (element_exists(By.CLASS_NAME, "price-1") == True):
                    price = driver.find_element(By.CLASS_NAME, "price-1").text
                    sale_price = driver.find_element(By.CLASS_NAME, "price-2").text
                else: 
                    sale_price = driver.find_element(By.CLASS_NAME, "price-2").text
                sale_price = driver.find_element(By.CLASS_NAME, "price-2").text
                picture = driver.find_element(By.XPATH, '//*[@id="mainImg"]').get_attribute('src')
                url = driver.find_element(By.XPATH, '/html/head/meta[8]').get_attribute('content')
                
                driver.find_element(By.CSS_SELECTOR, "#buyInfo > a").click()
                time.sleep(1)
                volume = driver.find_element(By.XPATH,'//*[@id="artcInfo"]/dl[2]/dd').text
                skin_type = driver.find_element(By.XPATH,'//*[@id="artcInfo"]/dl[3]/dd').text
                ingredient = driver.find_element(By.XPATH,'//*[@id="artcInfo"]/dl[8]/dd').text
                
                print(brand)
                print(name)
                print(price) 
                print(sale_price)
                print(picture)
                print(url)
                print(volume)
                print(skin_type)
                print(ingredient)
                
                #리뷰 크롤링
                driver.find_element(By.CSS_SELECTOR, "#reviewInfo > a").click() #리뷰버튼 클릭
                time.sleep(2) 
                driver.find_element(By.XPATH, '//*[@id="gdasSort"]/li[3]/a').click() #최신순 클릭
                time.sleep(1) 
                driver.find_element(By.CSS_SELECTOR, "#searchType_3").click() #체험단 체크 해제 
                time.sleep(1)
                
                review_count = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a/span').text 
                review_count = int(review_count.replace('(', '').replace(')', '').replace(',', ''))
           
                all_reviews = []
                
                #리뷰가 100이 안넘는다면 
                if (review_count < 100):
                    p = int(str(review_count)[0]) #십의자리 ex)리뷰수가 68개면 6만가져옴 >> 마지막페이지 알기위해 
                    k = 1
                    while k <= p:
                        reviews = [] #수정
                        for r in range(1, 11):
                            try:
                                review = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{r}]/div[2]/div[3]').text
                                reviews.append([review])
                            except:
                                break
                        all_reviews.extend(reviews)
                        
                        if (element_exists(By.XPATH, '//*[@id="sel_option_box"]') == True):
                            button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{k}]').click()   
                            k += 1
                            time.sleep(2)
                        else:
                            button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{k}]').click()
                            k += 1
                            time.sleep(2)
                    print(all_reviews)
                    Lotion_dataset.loc[len(Lotion_dataset)] = [brand, name,price,sale_price,picture,url,volume,skin_type,ingredient, all_reviews]    
                    driver.back()
                    
                # 리뷰가 100개 초과일 때    
                else:
                    k = 1
                    while k < 10: # k가 9까지일때가 10페이지임
                        reviews = []
                        for r in range(1, 11):
                            try:
                                review = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{r}]/div[2]/div[3]').text
                                reviews.append([review])
                            except:
                                break
                        all_reviews.extend(reviews)
                        
                        if (element_exists(By.XPATH, '//*[@id="sel_option_box"]') == True):
                            button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{k}]').click()   
                            k += 1
                            time.sleep(2)
                        else:
                            button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{k}]').click()
                            k += 1
                            time.sleep(2)
                    
                    print(all_reviews)        
                    
                    Lotion_dataset.loc[len(Lotion_dataset)] = [brand, name,price,sale_price,picture,url,volume,skin_type,ingredient, all_reviews]    
                    driver.back()
    #1에서 4페이지까지는 한페이지당 48개 상품이 있음
    else:            
        for i in range(2,14):
            for j in range(1, 5):
                driver.find_element(By.XPATH, f'//*[@id="Contents"]/ul[{i}]/li[{j}]/div/div/a').click()
                # driver.find_element(By.CLASS_NAME, 'prd_info').click()
                time.sleep(1) 
                brand = driver.find_element(By.CLASS_NAME, "prd_brand").text
                name = driver.find_element(By.CLASS_NAME, "prd_name").text
                if (element_exists(By.CLASS_NAME, "price-1") == True):
                    price = driver.find_element(By.CLASS_NAME, "price-1").text
                    sale_price = driver.find_element(By.CLASS_NAME, "price-2").text
                else: 
                    sale_price = driver.find_element(By.CLASS_NAME, "price-2").text
                picture = driver.find_element(By.XPATH, '//*[@id="mainImg"]').get_attribute('src')
                url = driver.find_element(By.XPATH, '/html/head/meta[8]').get_attribute('content')
                
                driver.find_element(By.CSS_SELECTOR, "#buyInfo > a").click()
                time.sleep(1)
                volume = driver.find_element(By.XPATH,'//*[@id="artcInfo"]/dl[2]/dd').text
                skin_type = driver.find_element(By.XPATH,'//*[@id="artcInfo"]/dl[3]/dd').text
                ingredient = driver.find_element(By.XPATH,'//*[@id="artcInfo"]/dl[8]/dd').text
                
                print(brand)
                print(name)
                print(price) 
                print(sale_price)
                print(picture)
                print(url)
                print(volume)
                print(skin_type)
                print(ingredient)

                    
                #리뷰 크롤링
                driver.find_element(By.CSS_SELECTOR, "#reviewInfo > a").click() #리뷰버튼 클릭
                time.sleep(2) 
                driver.find_element(By.XPATH, '//*[@id="gdasSort"]/li[3]/a').click() #최신순 클릭
                time.sleep(1) 
                driver.find_element(By.CSS_SELECTOR, "#searchType_3").click() #체험단 체크 해제 
                time.sleep(1)
                
                review_count = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a/span').text 
                review_count = int(review_count.replace('(', '').replace(')', '').replace(',', ''))
            
                all_reviews = []
                #리뷰가 100이 안넘는다면 
                if (review_count < 100):
                    
                    p = int(str(review_count)[0])
                    k = 1
                    while k <= p:
                        reviews = [] #수정
                        for r in range(1, 11):
                            try:
                                review = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{r}]/div[2]/div[3]').text
                                reviews.append([review])
                            except:
                                break
                        all_reviews.extend(reviews)
                    
                        if (element_exists(By.XPATH, '//*[@id="sel_option_box"]') == True):
                            button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{k}]').click()   
                            k += 1
                            time.sleep(2)
                        else:
                            button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{k}]').click()
                            k += 1
                            time.sleep(2)
                    
                    print(all_reviews)        
                    
                    Lotion_dataset.loc[len(Lotion_dataset)] = [brand, name,price,sale_price,picture,url,volume,skin_type,ingredient, all_reviews]    
                    driver.back()
                else:
                # 리뷰가 100개 초과일 때
                    k = 1
                    while k < 10:
                        reviews = []
                        for r in range(1, 11):
                            try:
                                review = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{r}]/div[2]/div[3]').text
                                reviews.append([review])
                            except:
                                break
                        all_reviews.extend(reviews)
                        
                        if (element_exists(By.XPATH, '//*[@id="sel_option_box"]') == True):
                            button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{k}]').click()   
                            k += 1
                            time.sleep(2)
                        else:
                            button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{k}]').click()
                            k += 1
                            time.sleep(2)
                    
                    print(all_reviews)        
                    
                    Lotion_dataset.loc[len(Lotion_dataset)] = [brand, name,price,sale_price,picture,url,volume,skin_type,ingredient, all_reviews]    
                    driver.back()

아이디얼포맨
아이디얼 포 맨 퍼펙트 올인원 HDEX콜라보 더블기획
48,000
원
18,850
원
https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0019/A00000019889002ko.jpg?l=ko
https://www.oliveyoung.co.kr/store/G.do?goodsNo=A000000198890
150ml+150ml+파우치(증정)
모든 피부 타입
정제수, 글리세린, 메틸트라이메티콘, 메틸프로판다이올, 에틸헥실스테아레이트, 에탄올, 1,2-헥산다이올, 나이아신아마이드, 하이드로제네이티드폴리(C6-14올레핀), 폴리글리세릴-3다이스테아레이트, 다시마추출물, 한련초잎추출물, 병풀추출물, 무화과추출물, 올레라케아아크멜라추출물, 당느릅나무뿌리추출물, 줄맨드라미씨추출물, 슈도알테로모나스발효추출물, 플랑크톤추출물, 하이드로제네이티드레시틴, 아르간커넬오일, 글리세릴스테아레이트, 트레할로오스, 알지닌, 카보머, 글리세릴스테아레이트시트레이트, 부틸렌글라이콜, 판테놀, 소듐스테아로일글루타메이트, 아데노신, 다이메티콘, 베헤닐알코올, 스쿠알란, 하이드로제네이티드폴리아이소부텐, 세테아릴알코올, 다이소듐이디티에이, 베타인, 프룩토올리고사카라이드, 아이소노닐아이소노나노에이트, 폴리글리세릴-10스테아레이트, 글리세릴스테아레이트에스이, 베타-글루칸, 스테아릭애씨드, 시어버터, 폴리글리세릴-2다이폴리하이드록시스테아레이트, 콜레스테롤, 하이드롤라이즈드하이알루로닉애씨드, 세라마이드엔피, 이눌린라우릴카바메이트, 잔탄검, 다이소듐스테아로일글루타메이트, 세린, 알라닌, 프롤린, 해수, 소듐포스페이트, 마데카소사이드, 소듐하이드록사이드, 아세틸헥사펩타이드-8, 에틸헥실글리세린, 카프릴릴글라이콜, 향료, 부틸페닐메틸프로피오날, 리모넨
[['피부가 약해서 타제품은 따갑지만 이제품은\n순해서 좋아요 감사합니다'], ['케이스가 검정색으로 바뀌었지만 이전이랑 내용물은 같은 것 같네요. 향도 은은하고 발림성도 좋아서 꾸준히 사용

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="gdasContentsArea"]/div/div[7]/a[2]"}
  (Session info: chrome=127.0.6533.72); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6DA199632+30946]
	(No symbol) [0x00007FF6DA14E3C9]
	(No symbol) [0x00007FF6DA046FDA]
	(No symbol) [0x00007FF6DA09822C]
	(No symbol) [0x00007FF6DA09850C]
	(No symbol) [0x00007FF6DA0DDCB7]
	(No symbol) [0x00007FF6DA0BCAAF]
	(No symbol) [0x00007FF6DA0DB041]
	(No symbol) [0x00007FF6DA0BC813]
	(No symbol) [0x00007FF6DA08A6E5]
	(No symbol) [0x00007FF6DA08B021]
	GetHandleVerifier [0x00007FF6DA2CF83D+1301229]
	GetHandleVerifier [0x00007FF6DA2DBDB7+1351783]
	GetHandleVerifier [0x00007FF6DA2D2A03+1313971]
	GetHandleVerifier [0x00007FF6DA1CDD06+245686]
	(No symbol) [0x00007FF6DA15758F]
	(No symbol) [0x00007FF6DA153804]
	(No s

In [26]:
import time
import json
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ['enable-logging'])
chrome_options.add_argument("--disable-blink-features-AutomationControlled")

driver = webdriver.Chrome(options=chrome_options)

# 브라우저 창 크기 설정
driver.set_window_size(1920, 1080)

Lotion_dataset = pd.DataFrame(columns=['brand', 'name', 'price', 'sale_price', 'picture', 'url', 'volume', 'skin_type', 'ingredient', 'review'])

def element_exists(by, value):
    try:
        driver.find_element(by, value)
        return True
    except NoSuchElementException:
        return False

def save_checkpoint(checkpoint):
    with open('checkpoint.json', 'w') as f:
        json.dump(checkpoint, f)

def load_checkpoint():
    if os.path.exists('checkpoint.json'):
        with open('checkpoint.json', 'r') as f:
            return json.load(f)
    else:
        return {'page_num': 1, 'i': 2, 'j': 1, 'k': 1, 'review_page': 1, 'review_part': 'normal'}

checkpoint = load_checkpoint()

def update_checkpoint(page_num=None, i=None, j=None, k=None, review_page=None, review_part=None):
    if page_num is not None:
        checkpoint['page_num'] = page_num
    if i is not None:
        checkpoint['i'] = i
    if j is not None:
        checkpoint['j'] = j
    if k is not None:
        checkpoint['k'] = k
    if review_page is not None:
        checkpoint['review_page'] = review_page
    if review_part is not None:
        checkpoint['review_part'] = review_part
    save_checkpoint(checkpoint)

def process_product_page(brand, name, price, sale_price, picture, url, volume, skin_type, ingredient):
    # 리뷰 크롤링
    review_count = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a/span').text
    review_count = int(review_count.replace('(', '').replace(')', '').replace(',', ''))
    
    driver.find_element(By.CSS_SELECTOR, "#reviewInfo > a").click() # 리뷰버튼 클릭
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="gdasSort"]/li[3]/a').click() # 최신순 클릭
    time.sleep(1)
    if (review_count >= 100):
        driver.find_element(By.CSS_SELECTOR, "#searchType_3").click() # 체험단 체크 해제
        time.sleep(1)

    all_reviews = []
    k = checkpoint['review_page']

    if (review_count < 100):
        p = int(str(review_count)[0])  # 십의 자리
        while k <= p:
            reviews = []
            for r in range(1, 11):
                if k == p:
                    for r in range(1, review_count % 10 + 1):
                        review = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{r}]/div[2]/div[3]').text
                        reviews.append([review])   
                    all_reviews.extend(reviews)         
                    driver.back()
                    
                review = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{r}]/div[2]/div[3]').text
                reviews.append([review])
            all_reviews.extend(reviews)
            
            # 다음 페이지 버튼 누르기
            if element_exists(By.XPATH, '//*[@id="sel_option_box"]'):
                button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{k}]').click()
            else:
                button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{k}]').click()

            k += 1
            update_checkpoint(review_page=k)
            time.sleep(2)

    else:
        while k < 10:
            reviews = []
            for r in range(1, 11):
                try:
                    review = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{r}]/div[2]/div[3]').text
                    reviews.append([review])
                except:
                    break
            all_reviews.extend(reviews)

            if element_exists(By.XPATH, '//*[@id="sel_option_box"]'):
                button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{k}]').click()
            else:
                button = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{k}]').click()

            k += 1
            update_checkpoint(review_page=k)
            time.sleep(2)

    Lotion_dataset.loc[len(Lotion_dataset)] = [brand, name, price, sale_price, picture, url, volume, skin_type, ingredient, all_reviews]
    print(all_reviews)

for page_num in range(checkpoint['page_num'], 6):
    update_checkpoint(page_num=page_num)
    olive_url = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010016&fltDispCatNo=&prdSort=01&pageIdx={page_num}&rowsPerPage=48&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=1&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat100000100010016_Small&amplitudePageGubun=&t_page=&t_click=&midCategory=%EB%A1%9C%EC%85%98&smallCategory=%EC%A0%84%EC%B2%B4&checkBrnds=&lastChkBrnd="
    driver.get(olive_url)
    time.sleep(1)

    # 5페이지는 42개상품밖에 없음
    if page_num == 5:
        for i in range(checkpoint['i'], 13):
            update_checkpoint(i=i)
            if i == 12:
                for j in range(checkpoint['j'], 3):
                    update_checkpoint(j=j)
                    driver.find_element(By.XPATH, f'//*[@id="Contents"]/ul[{i}]/li[{j}]/div/div/a').click()
                    time.sleep(1)
                    brand = driver.find_element(By.CLASS_NAME, "prd_brand").text
                    name = driver.find_element(By.CLASS_NAME, "prd_name").text
                    if element_exists(By.CLASS_NAME, "price-1"):
                        price = driver.find_element(By.CLASS_NAME, "price-1").text
                        sale_price = driver.find_element(By.CLASS_NAME, "price-2").text
                    else:
                        sale_price = driver.find_element(By.CLASS_NAME, "price-2").text
                    picture = driver.find_element(By.XPATH, '//*[@id="mainImg"]').get_attribute('src')
                    url = driver.find_element(By.XPATH, '/html/head/meta[8]').get_attribute('content')

                    driver.find_element(By.CSS_SELECTOR, "#buyInfo > a").click()
                    time.sleep(1)
                    volume = driver.find_element(By.XPATH, '//*[@id="artcInfo"]/dl[2]/dd').text
                    skin_type = driver.find_element(By.XPATH, '//*[@id="artcInfo"]/dl[3]/dd').text
                    ingredient = driver.find_element(By.XPATH, '//*[@id="artcInfo"]/dl[8]/dd').text

                    print(brand)
                    print(name)
                    print(price)
                    print(sale_price)
                    print(picture)
                    print(url)
                    print(volume)
                    print(skin_type)
                    print(ingredient)

                    process_product_page(brand, name, price, sale_price, picture, url, volume, skin_type, ingredient)
                    update_checkpoint(review_page=1)
                    driver.back()
            else:
                for j in range(checkpoint['j'], 4):
                    update_checkpoint(j=j)
                    driver.find_element(By.XPATH, f'//*[@id="Contents"]/ul[{i}]/li[{j}]/div/div/a').click()
                    time.sleep(1)
                    brand = driver.find_element(By.CLASS_NAME, "prd_brand").text
                    name = driver.find_element(By.CLASS_NAME, "prd_name").text
                    if element_exists(By.CLASS_NAME, "price-1"):
                        price = driver.find_element(By.CLASS_NAME, "price-1").text
                        sale_price = driver.find_element(By.CLASS_NAME, "price-2").text
                    else:
                        sale_price = driver.find_element(By.CLASS_NAME, "price-2").text
                    picture = driver.find_element(By.XPATH, '//*[@id="mainImg"]').get_attribute('src')
                    url = driver.find_element(By.XPATH, '/html/head/meta[8]').get_attribute('content')

                    driver.find_element(By.CSS_SELECTOR, "#buyInfo > a").click()
                    time.sleep(1)
                    volume = driver.find_element(By.XPATH, '//*[@id="artcInfo"]/dl[2]/dd').text
                    skin_type = driver.find_element(By.XPATH, '//*[@id="artcInfo"]/dl[3]/dd').text
                    ingredient = driver.find_element(By.XPATH, '//*[@id="artcInfo"]/dl[8]/dd').text

                    print(brand)
                    print(name)
                    print(price)
                    print(sale_price)
                    print(picture)
                    print(url)
                    print(volume)
                    print(skin_type)
                    print(ingredient)

                    process_product_page(brand, name, price, sale_price, picture, url, volume, skin_type, ingredient)
                    update_checkpoint(review_page=1)
                    driver.back()
    else:
        for i in range(checkpoint['i'], 14):
            update_checkpoint(i=i)
            for j in range(checkpoint['j'], 5):
                update_checkpoint(j=j)
                driver.find_element(By.XPATH, f'//*[@id="Contents"]/ul[{i}]/li[{j}]/div/div/a').click()
                time.sleep(1)
                brand = driver.find_element(By.CLASS_NAME, "prd_brand").text
                name = driver.find_element(By.CLASS_NAME, "prd_name").text
                if element_exists(By.CLASS_NAME, "price-1"):
                    price = driver.find_element(By.CLASS_NAME, "price-1").text
                    sale_price = driver.find_element(By.CLASS_NAME, "price-2").text
                else:
                    sale_price = driver.find_element(By.CLASS_NAME, "price-2").text
                picture = driver.find_element(By.XPATH, '//*[@id="mainImg"]').get_attribute('src')
                url = driver.find_element(By.XPATH, '/html/head/meta[8]').get_attribute('content')

                driver.find_element(By.CSS_SELECTOR, "#buyInfo > a").click()
                time.sleep(1)
                volume = driver.find_element(By.XPATH, '//*[@id="artcInfo"]/dl[2]/dd').text
                skin_type = driver.find_element(By.XPATH, '//*[@id="artcInfo"]/dl[3]/dd').text
                ingredient = driver.find_element(By.XPATH, '//*[@id="artcInfo"]/dl[8]/dd').text

                print(brand)
                print(name)
                print(price)
                print(sale_price)
                print(picture)
                print(url)
                print(volume)
                print(skin_type)
                print(ingredient)

                process_product_page(brand, name, price, sale_price, picture, url, volume, skin_type, ingredient)
                update_checkpoint(review_page=1)
                driver.back()


포맨트
포맨트 시그니처 올인원 에센스 96ml
28,900
원
23,100
원
https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0015/A00000015599406ko.jpg?l=ko
https://www.oliveyoung.co.kr/store/G.do?goodsNo=A000000155994
Day 48 ml / Night 48 ml (48ml l e 1.62 fl. oz.*2ea)
모든피부용
정제수, 에칠헥실메톡시신나메이트, 호모살레이트, 비스-에칠헥실옥시페놀메톡시페닐트리아진, 다이프로필렌글라이콜, 다이페닐실록시페닐트라이메티콘, 에칠헥실살리실레이트, 티타늄디옥사이드, 나이아신아마이드, 글리세릴스테아레이트, 부틸옥틸살리실레이트, 폴리글리세릴-2스테아레이트, 스테아릭애씨드, 스테아릴알코올, 부틸렌글라이콜, 사이클로펜타실록세인, 솔비탄세스퀴올리에이트, 아크릴레이트/C10-30알킬아크릴레이트크로스폴리머, 카프릴릴글라이콜, 소듐메틸스테아로일타우레이트, 트로메타민, 펜틸렌글라이콜, 하이드록시아세토페논, 아크릴레이트/에틸헥실아크릴레이트/다이메티콘메타크릴레이트코폴리머, 향료, 트레할로오스, 트라이에톡시실릴에틸폴리다이메틸실록시에틸헥실다이메티콘, 에틸헥실글리세린, 아데노신, 다이소듐이디티에이, 1,2-헥산다이올, 병풀추출물, 아라키딕애씨드, 황금추출물, 호장근뿌리추출물, 팔미틱애씨드, 소듐하이알루로네이트, 트루로즈오브예리코추출물, 스페인감초뿌리추출물, 녹차추출물, 올레익애씨드, 로즈마리잎추출물, 마트리카리아꽃추출물, 대왕송잎추출물, 칡뿌리추출물, 달맞이꽃추출물, 당느릅나무뿌리추출물, 하이드롤라이즈드하이알루로닉애씨드, 하이드록시프로필트라이모늄하이알루로네이트, 소듐하이알루로네이트크로스폴리머, 소듐아세틸레이티드하이알루로네이트, 시트로넬올, 제라니올, 헥실신남알, 리날룰,
[['하나만 써도 되니깐 간편하거 피부가 좋아진게 느껴집니다 또 구매 할거같아요'], ['남자친구가 쓰는 로션이에요 선크림이랑 로션이 같이 있어서

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="gdasList"]/li[1]/div[2]/div[3]"}
  (Session info: chrome=127.0.6533.72); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6DA199632+30946]
	(No symbol) [0x00007FF6DA14E3C9]
	(No symbol) [0x00007FF6DA046FDA]
	(No symbol) [0x00007FF6DA09822C]
	(No symbol) [0x00007FF6DA09850C]
	(No symbol) [0x00007FF6DA0DDCB7]
	(No symbol) [0x00007FF6DA0BCAAF]
	(No symbol) [0x00007FF6DA0DB041]
	(No symbol) [0x00007FF6DA0BC813]
	(No symbol) [0x00007FF6DA08A6E5]
	(No symbol) [0x00007FF6DA08B021]
	GetHandleVerifier [0x00007FF6DA2CF83D+1301229]
	GetHandleVerifier [0x00007FF6DA2DBDB7+1351783]
	GetHandleVerifier [0x00007FF6DA2D2A03+1313971]
	GetHandleVerifier [0x00007FF6DA1CDD06+245686]
	(No symbol) [0x00007FF6DA15758F]
	(No symbol) [0x00007FF6DA153804]
	(No symbo

In [27]:
Lotion_dataset

,brand,name,price,sale_price,picture,url,volume,skin_type,ingredient,review
0,포맨트,포맨트 시그니처 올인원 에센스 96ml,"28,900\n원","23,100\n원",https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/G.do?goodsN...,Day 48 ml / Night 48 ml (48ml l e 1.62 fl. oz....,모든피부용,"정제수, 에칠헥실메톡시신나메이트, 호모살레이트, 비스-에칠헥실옥시페놀메톡시페닐트리아...","[[하나만 써도 되니깐 간편하거 피부가 좋아진게 느껴집니다 또 구매 할거같아요], ..."
1,아이디얼포맨,[시카]아이디얼 포 맨 시카 올인원 기획 2024 AD(150mL+50mL),"30,000\n원","21,500\n원",https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/G.do?goodsN...,150mL+50ml(증정),모든 피부 타입,"병풀잎추출물, 정제수, 프로판다이올, 에틸헥실스테아레이트, 나이아신아마이드, 메틸트...","[[피부에 자극없이 순하고 수분감을 채워주어 여름에 쓰기도 좋아요], [올인원이라 ..."
2,닥터지,닥터지 레드 블레미쉬 포 맨 올인원 크림 150ml,"29,700\n원","19,000\n원",https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/G.do?goodsN...,닥터지 레드 블레미쉬 포 맨 올인원 크림 150ml,민감 피부 / 건성 피부,"정제수, 프로판다이올, 부틸렌글라이콜다이카프릴레이트/다이카프레이트, 메틸프로판다이올...",[[안 끈적이는 걸 원해서 매장에서 발라보고 샀는데 피부가 보들보들해 지는 것 같아...
